In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

I. Функция, вычленяющая 'Окончание ссылки для итерации постоянной итерации'

In [2]:
def extract_link_pattern(inital_url):
    '''Функция возвращает ссылку, на следующую страницу (next) содержащую новости'''
    
    response = requests.get(inital_url).text

    html_structure = BeautifulSoup(response, 'html.parser')

    next_button = html_structure.findAll('a', class_= 'btn lenta_pager_next') #Содержит HTML-тэг, в формате строки

    pattern = r"\?down=\d+"

    try:
        return re.findall(pattern, str(next_button[0]))[0]
    except:
        return 0

II. Цикл, формирующий список, состоящий из всех страниц новостей по конкретной тематике

In [3]:
initial_url = 'https://vtomske.ru/tag/economics' #Начальная ссылка

string_to_be_formatted = 'https://vtomske.ru/tag/economics{}'

links_list = []

links_list.append(initial_url)

#while len(links_list) <= 5000:
while True:

    next_pattern = extract_link_pattern(initial_url) #Переменная содержит в себе паттерн продолжения

    #print(next_pattern)

    if next_pattern == 0:
        break #Цикл прекращается, если на странице отсутсвует кнопка 'Раньше' == Новости по тематике закончились.

    next_link = string_to_be_formatted.format(next_pattern) #Форматируем строку - ссылку, добавляя в нее продолжение

    links_list.append(next_link) #Сохраняем ссылку в список ссылок

    initial_url = next_link

In [4]:
def form_link_list(initial_url):
    '''Функция получает начальную ссылку на новостную тематику с сайта vtomske.ru в след. формате:
    https://vtomske.ru/tag/russia
    И возвращает список ссылок на все страницы новостей по данной новостной тематике.'''
    
    #string_to_be_formatted = 'https://vtomske.ru/tag/economics{}'
    string_to_be_formatted = f"{initial_url}{{}}"

    links_list = []

    links_list.append(initial_url)

#while len(links_list) <= 5000:
    while True:
        next_pattern = extract_link_pattern(initial_url) #Переменная содержит в себе паттерн продолжения

        if next_pattern == 0:
            break #Цикл прекращается, если на странице отсутсвует кнопка 'Раньше' == Новости по тематике закончились.

        next_link = string_to_be_formatted.format(next_pattern) #Форматируем строку - ссылку, добавляя в нее продолжение

        links_list.append(next_link) #Сохраняем ссылку в список ссылок

        initial_url = next_link

    return links_list

III. Попытка получать именно новости с ссылок

Опишем функцию для получения ссылок на конкретные новости со страницы

In [5]:
def extract_news_links(url):
    '''Функция, позволяющая извлечь список URL для всех новостей, находящихся на странице.
    Возвращает список (python list), состоящий из завершенных URL'''
    response = requests.get(url).text

    soup = BeautifulSoup(response, 'html.parser')

    raw_list = soup.findAll('a', class_="lenta_material") #Содержит в список, из которого следует извлечь ссылку на новость

    extract_pattern = r'href="(.*?)"' #паттерн регулярного выражения, позволяющий выделить окончание новости

    urls_final_list = []

    for tag in raw_list:
        
        tag = str(tag)

        result = re.findall(extract_pattern, tag)

        urls_final_list.append(result[0])

    return urls_final_list
    #return link_list

IV. Отлично, извлекаем заголовок и текст новости из новости

In [6]:
def extract_title_and_text(news_url):
    '''Функция извлекает следующий аспекты из новостной статьи:
    1. Новостной заголовок.
    2. Текст новостной статьи.
    3. Дату публикации.
    4. Тэги.
    На вход получает ссылку на конкретную новость.'''

    start_link = 'https://vtomske.ru'

    full_link_url = start_link + news_url

    beautiful_html_structure = BeautifulSoup(requests.get(full_link_url).text, 'html.parser')

    headline = beautiful_html_structure.findAll('h1')[0]

    article_text = beautiful_html_structure.findAll('p')

    #Извлечение даты

    publication_date = beautiful_html_structure.findAll('time', class_="material-date")

    date_pattern = r'\d{4}-\d{2}-\d{2}'

    date = re.findall(date_pattern, str(publication_date))[0]

    #/Извлечение даты

    #Извлечение тегов

    tags_raw = beautiful_html_structure.findAll('div', class_="material-tags")

    tags_pattern = r">\w+<"

    tags = re.findall(tags_pattern, str(tags_raw))

    for index in range(0, len(tags)):
        tags[index] = tags[index].replace(">", "").replace("<", "")

    #/Излвечение тэгов

    #Возвращаем результат

    final_dict = {
        'Заголовок': str(headline),
        'Текст новости': str(article_text),
        'Дата публикации': str(date),
        'Тэги': str(tags)
    }
    
    return final_dict


VI. Опишем функцию парсинга

In [11]:
def parse(theme_url):
    '''Функция, осуществляющая парсинг конкретной тематике с сайта vtomske.ru.
    Ссылка должна вести на существующую тематику и иметь ввиду, типа:
    https://vtomske.ru/tag/russia
    Возвращает датафрейм и сохраняет его в активную директорию'''
    news_pages = form_link_list(theme_url)

    print(f"По выбранной тематике на сайте представлено {len(news_pages)} страниц новостей")

    print('Процесс сбора новостей запущен...\nПрогресс обработки страниц:')

    information_list = [] #Лист, содержащий в себе информацию по каждой новости

    for page in tqdm(news_pages):
        news_list = extract_news_links(page) #Список, содержащий в себе ссылки на конкретные новости с одной страницы.

        for news in news_list:
            news_info = extract_title_and_text(news)
            information_list.append(news_info)
    
    final_df = pd.DataFrame(information_list)

    final_df.to_csv(f'all_news_for_{theme_url}.csv')

    return final_df

In [ ]:
test_link = 'https://vtomske.ru/tag/economics'

new_df = parse(test_link)